<a href="https://colab.research.google.com/github/Eng-Abdelrahman-Mostafa-Mohamed/AbdelrahmanMostafa.github.io/blob/main/ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Load Data

In [19]:
import tensorflow as tf

In [20]:
#loading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Subset Class 0 and Class 1 from Data

In [21]:
# subsetting for classes 0,1 only

x_train = x_train[np.logical_or(y_train == 0, y_train == 1)]
y_train = y_train[np.logical_or(y_train == 0, y_train == 1)]

x_test = x_test[np.logical_or(y_test == 0, y_test == 1)]
y_test = y_test[np.logical_or(y_test == 0, y_test == 1)]


In [22]:
np.unique(y_train), np.unique(y_test)

(array([0, 1], dtype=uint8), array([0, 1], dtype=uint8))

In [23]:
x_train=x_train.reshape(len(x_train),-1)

In [24]:
x_test=x_test.reshape(len(x_test),-1)

In [25]:
x_train.shape[0]

12665

Standardize Data

In [26]:
x_train=(x_train-np.mean(x_train,axis=0))/(np.std(x_train,axis=0)+10e-16)
x_test=(x_test-np.mean(x_test,axis=0))/(np.std(x_test,axis=0)+10e-16)

Divide data into training and validation set

In [49]:
X_train=x_train[0:10132]
Y_train=y_train[0:10132]
X_validation=x_train[10132:10132+2533]
Y_validation=y_train[10132:10132+2533]

In [50]:
epochs=100
eta=0.1
tolerance = 10e-8
Lambda=[0.01,0.1]
#batch_size=10
batch_size=[100,200]
m = X_train.shape[0]
ACCURACY=[]
Name=[]

Sigmoid Function

In [51]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

Cost Function

In [52]:
def cost_function(phiz, y_train):
    return (y_train * np.log(phiz) - (1 - y_train) * np.log(1 - phiz)).mean()

Cost Function with L1 Regularization(lasso)

In [53]:
def cost_function_L1(phiz, Y_train,w,Lambda):
    return ((Y_train * np.log(phiz) - (1 - Y_train) * np.log(1 - phiz)).mean())+0.5*Lambda*np.mean(np.abs(w))

Accuracy Function

In [54]:
def accuracy(x,y,w,b):
  z=np.dot(x,w.T)+b
  y_pred =sigmoid(z)
  for i in range(len(y_pred)):
    if (y_pred[i]>0.5):
     y_pred[i]=1
    else:
      y_pred[i]=0
  accuracy=np.mean(np.equal(y_pred, y))*100

  return accuracy

Gradient Descent Function

In [55]:
def Gradient_Descent(X_train,Y_train,w,b,epochs,eta,Lambda):
    for i in range (epochs):
          z=np.dot(X_train,w.T)+b
          phiZ = sigmoid(z)
          Cost[i] =cost_function_L1(phiZ, Y_train,w,Lambda)
          w = w - (eta * (np.dot((phiZ - Y_train).T , X_train) /len(X_train)+Lambda/len(X_train)*np.sign(w)))
          b = b - eta * np.mean(phiZ - Y_train)
          if (Cost[i]<tolerance):
            break;
     
    return w,b

In [56]:
for lamb in range(len(Lambda)):
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  w,b=Gradient_Descent(X_train,Y_train,w,b,epochs,eta,Lambda[lamb])
  ACCuracy=accuracy(X_train,Y_train,w,b)
  print('Accuracy  for train with Lambda (',Lambda[lamb],') is =',ACCuracy)
  Accuracy=accuracy(X_validation,Y_validation,w,b)
  print('Accuracy for validation with Lambda (',Lambda[lamb],') is =',Accuracy)
  print('---------------------------------------------------------------------')

Accuracy  for train with Lambda ( 0.01 ) is = 98.5491512041058
Accuracy for validation with Lambda ( 0.01 ) is = 98.61823924200553
---------------------------------------------------------------------
Accuracy  for train with Lambda ( 0.1 ) is = 98.56889064350572
Accuracy for validation with Lambda ( 0.1 ) is = 98.53928148440583
---------------------------------------------------------------------


In [57]:
ACCURACY.append(Accuracy)
Name.append('Gradient_Descent')
Accuracy

98.53928148440583

Mini_Batch_Gradient_Descent Function

In [58]:
def mini_batch_gradient_descent(X_train, Y_train,w,b,eta,batch_size):
  m=X_train.shape[0]
  n=m/batch_size
  n=int(n)
  for i in range(epochs):
    for j in range(0,m,n):
     X_batch=X_train[j:j+n]
     Y_batch=Y_train[j:j+n]
     z=np.dot(X_batch,w.T)+b
     phiZ = sigmoid(z)
     Cost[i] =cost_function(phiZ, Y_batch)
     w = w - (eta * np.dot((phiZ - Y_batch).T , X_batch)) /len(X_batch)
     b = b - eta * np.mean(phiZ - Y_batch)
     if (Cost[i]<tolerance):
      break;
      
  return w,b

 

for i in range(len(batch_size)):
  print('this is the ',i+1,'Batch')
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  w,b=mini_batch_gradient_descent(X_train, Y_train,w,b,eta,batch_size[i])
  ACCuracy=accuracy(X_train,Y_train,w,b)
  print('Accuracy for Train is =',ACCuracy)   
  Accuracy=accuracy(X_validation,Y_validation,w,b)
  print('Accuracy for Validation is =',Accuracy)   


 

this is the  1 Batch
Accuracy for Train is = 99.98026056060007
Accuracy for Validation is = 99.68416896960126
this is the  2 Batch
Accuracy for Train is = 100.0
Accuracy for Validation is = 99.56573233320174


In [59]:
Accuracy=accuracy(X_validation,Y_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('Mini_Batch_Gradient_Descent')
Accuracy

99.56573233320174

RMS_PROP Optimizer Function

In [60]:
def rms_prop(X_train, Y_train, w, b, eta, beta, epsilon,batch_size):
  vdw=np.zeros_like(w)
  vdb=np.zeros_like(b)
  m=X_train.shape[0]
  n=m/batch_size
  n=int(n)
  for i in range(epochs):
    for j in range(0,m,n):
     X_batch=X_train[j:j+n]
     Y_batch=Y_train[j:j+n]
     z=np.dot(X_batch,w.T)+b
     phiZ = sigmoid(z)
     dW = (np.dot((phiZ - Y_batch).T , X_batch)) /len(X_batch)
     db=np.mean(phiZ - Y_batch)
     #let assume or initialize v_dw by 0 or np.zeros(len(w))
     vdw = beta*vdw+(1 - beta) * np.power(dW, 2)
     vdb=beta*vdb+(1 - beta) * np.power(db, 2)
     w= w-(eta / np.sqrt(vdw + epsilon)) * dW
     b= b-(eta / np.sqrt(vdb + epsilon)) * db
  return w,b

     
for i in range(len(batch_size)):
  print('this is the ',i+1,'Batch')
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  beta=0.9
  epsilon=10e-8
  w,b=rms_prop(X_train, Y_train,w,b,eta,beta,epsilon,batch_size[i])
  ACCuracy=accuracy(X_train,Y_train,w,b)
  print('Accuracy for Train is =',ACCuracy)   
  Accuracy=accuracy(X_validation,Y_validation,w,b)
  print('Accuracy for Validation is =',Accuracy)   

this is the  1 Batch
Accuracy for Train is = 99.90130280300039
Accuracy for Validation is = 99.36833793920252
this is the  2 Batch
Accuracy for Train is = 99.63482037110147
Accuracy for Validation is = 99.17094354520331


In [61]:
Accuracy=accuracy(X_validation,Y_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('RMSPROP Optimizer')
Accuracy

99.17094354520331

ADAM Oprimizer Function

In [63]:
def adam_optimizer(X_train, Y_train, w, b, eta,beta1, beta2,epsilon,t,batch_size):
      m=X_train.shape[0]
      n=m/batch_size
      n=int(n)
      v_dw_RMS = np.zeros_like(w)
      v_db_RMS = np.zeros_like(b)
      v_dw_momentum = np.zeros_like(w)
      v_db_momentum = np.zeros_like(b)                                               
      epsilon = epsilon
      for i in range(epochs):
          indices = np.random.permutation(X_train.shape[0])
          x = X_train[indices]
          y = Y_train[indices]
          for j in range(0, m, n):
              x_batch = x[j:j+n]
              y_batch = y[j:j+n]
              z = np.dot(x_batch, w) + b
              y_pred = 1 / (1 + np.exp(-z))
              dz = y_pred - y_batch
              dw = np.dot(x_batch.T, dz) / batch_size
              db = np.sum(dz) / batch_size
              # RMSprop part of velocity update
              v_dw_RMS = beta2 * v_dw_RMS + (1 - beta2) * (dw**2)
              v_db_RMS = beta2 * v_db_RMS + (1 - beta2) * (db**2)
              # momentum part of velocity update
              v_dw_momentum = beta1 * v_dw_momentum + (1 - beta1) * dw
              v_db_momentum = beta1 * v_db_momentum + (1 - beta1) * db
              # combine to make adam weights and bias 
              w -= eta * (v_dw_momentum / (np.sqrt(v_dw_RMS) + epsilon))
              b -= eta * (v_db_momentum / (np.sqrt(v_db_RMS) + epsilon))
      return w, b



for i in range(len(batch_size)):
  w=np.random.rand(784)
  b=np.random.rand(1)
  Cost=np.zeros(epochs)
  beta1=0.9
  beta2=0.9
  epsilon=10e-8
  t=1
  print('this is the ',i+1,'Batch')
  w,b=adam_optimizer(X_train, Y_train, w,b, 0.01, beta1, beta2, epsilon, t,batch_size[i])
  ACCuracy=accuracy(X_train,Y_train,w,b)
  print('Accuracy for Train is =',ACCuracy)   
  Accuracy=accuracy(X_validation,Y_validation,w,b)
  print('Accuracy for Validation is =',Accuracy)   

this is the  1 Batch
Accuracy for Train is = 99.8026056060008
Accuracy for Validation is = 99.36833793920252
this is the  2 Batch
Accuracy for Train is = 99.59534149230161
Accuracy for Validation is = 99.249901302803


In [64]:
Accuracy=accuracy(X_validation,Y_validation,w,b)
ACCURACY.append(Accuracy)
Name.append('adam Optimizer')
Accuracy

99.249901302803

In [65]:
ACCURACY

[98.53928148440583, 99.56573233320174, 99.17094354520331, 99.249901302803]

In [66]:
# Test with same data and look at the accuracy
z=np.dot(x_test,w.T)+b
y_pred =sigmoid(z)
for i in range(len(y_pred)):
  if (y_pred[i]>0.5):
     y_pred[i]=1
  else:
     y_pred[i]=0
  
    
accuracy=np.mean(np.equal(y_pred, y_test))*100
accuracy

99.33806146572104

In [67]:
name=pd.DataFrame(Name,columns=['Optimizer'])
ACCuracy=pd.DataFrame(ACCURACY,columns=['Accuracy'])

In [68]:
Table=pd.concat([name,ACCuracy],axis=1)
Table

,Optimizer,Accuracy
0,Gradient_Descent,98.539281
1,Mini_Batch_Gradient_Descent,99.565732
2,RMSPROP Optimizer,99.170944
3,adam Optimizer,99.249901


Use L1 regularization with gradient descent optimizer: This involves adding an L1 penalty term to the loss function for your logistic regression model, which encourages sparsity in the model weights. You can train this regularized model using gradient descent optimization, with a learning rate and regularization strength hyperparameters.



Use mini-batch gradient descent optimizer: This involves dividing your dataset into multiple batches, and updating the model weights using the gradients computed on each batch. You can train this model using a variety of batch sizes and learning rates, and evaluate its performance on the validation set.


Use RMSProp optimizer and Adam optimizer: These are both adaptive optimization algorithms that adjust the learning rate based on the magnitude of the gradients and the history of the parameter updates. You can train your logistic regression model using either of these optimizers, and tune the hyperparameters to obtain the best performance on the validation set.

L1 regularization with gradient descent optimizer: The L1 regularization can help to reduce overfitting and improve the generalization performance of the model. The choice of the regularization strength hyperparameter can affect the sparsity of the model weights and its performance on the validation set.

Mini-batch gradient descent optimizer: The mini-batch gradient descent can speed up the optimization process and reduce the variance of the gradient estimates. The choice of the batch size hyperparameter can affect the convergence rate and stability of the optimization algorithm.

RMSProp optimizer: The RMSProp optimizer can adaptively adjust the learning rate based on the magnitude of the gradients and the history of the parameter updates. It can converge faster than gradient descent and mini-batch gradient descent for certain types of problems.

Adam optimizer: The Adam optimizer combines ideas from momentum and RMSProp and is considered a state-of-the-art optimization algorithm. It can converge faster than other optimization algorithms, particularly for large datasets and complex models. However, it may be computationally expensive due to the additional computation required for maintaining the moving averages and may not always converge to the optimal solution.

The results of the implementation may vary depending on the random seed, but generally, we can see that the accuracy on the validation set is highest for the Adam optimizer, followed by the RMS Prop optimizer. The mini-batch gradient descent optimizer also performs well, with similar accuracy to the RMS Prop optimizer. The  gradient descent optimizer performs the worst in terms of accuracy.

In conclusion, we observed that the Adam optimizer performed the best in terms of accuracy on the validation set. The RMS Prop optimizer and mini-batch gradient descent optimizer also performed well, while the standard gradient descent optimizer performed the worst. The choice of lambda value also had a significant impact on the performance, with smaller lambda values generally resulting in better accuracy.